<a href="https://colab.research.google.com/github/mikeroguez/experimento/blob/main/Merge_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importar bibliotecas necesarias
import pandas as pd

# Cargar los datasets desde archivos locales
assignments_path = "/content/drive/MyDrive/Doctorado/Experimentos/Datos/DSv4.0/1. Para tratar/tareas/tareas_consolidado_cleaned_sin_cursos_reutilizados.csv"
exams_path = "/content/drive/MyDrive/Doctorado/Experimentos/Datos/DSv4.0/1. Para tratar/examenes/examenes_consolidados.csv"
access_path = "/content/drive/MyDrive/Doctorado/Experimentos/Datos/DSv4.0/1. Para tratar/accesos/accesos_consolidado_cleaned.csv"
forums_path = "/content/drive/MyDrive/Doctorado/Experimentos/Datos/DSv4.0/1. Para tratar/foros/foros_consolidado.csv"

assignments_df = pd.read_csv(assignments_path)
exams_df = pd.read_csv(exams_path)
access_df = pd.read_csv(access_path)
forums_df = pd.read_csv(forums_path)

# 1. Integrar "Exams" a "Assignments" basado en groupKey y email
merged_df = pd.merge(assignments_df, exams_df, how="left", on=["groupKey", "email"])

# 2. Integrar "Access" a la base intermedia usando curso, groupKey y email
merged_df = pd.merge(merged_df, access_df, how="left", on=["curso", "groupKey", "email"])

# 3. Integrar "Forums" a la base intermedia usando curso, groupKey y email
merged_df = pd.merge(merged_df, forums_df, how="left", on=["curso", "groupKey", "email"])

# 4. Exportar el dataset consolidado
output_path = "/content/drive/MyDrive/Doctorado/Experimentos/Datos/DSv4.0/2. Para unir/merged_dataset.csv"
merged_df.to_csv(output_path, index=False)

print(f"Dataset consolidado guardado en {output_path}")

Dataset consolidado guardado en /content/drive/MyDrive/Doctorado/Experimentos/Datos/DSv4.0/2. Para unir/merged_dataset.csv


**2. Limpieza de examenes, accesos y foros**

In [3]:
import pandas as pd

# Definir rutas de entrada y salida
input_path = "/content/drive/MyDrive/Doctorado/Experimentos/Datos/DSv4.0/2. Para unir/merged_dataset.csv"
output_path = "/content/drive/MyDrive/Doctorado/Experimentos/Datos/DSv4.0/2. Para unir/merged_cleaned_dataset.csv"

# Cargar el dataset
df = pd.read_csv(input_path)

### 1. MANEJO DE EXÁMENES

# Contar estudiantes con datos de exámenes por curso
exam_distribution = df.groupby("groupKey")["total_assigned_exams"].count()
total_students_per_group = df.groupby("groupKey")["email"].count()

# Identificar los casos de exámenes
courses_without_exams = exam_distribution[exam_distribution == 0].index  # Caso 1
courses_with_all_exams = exam_distribution[exam_distribution == total_students_per_group].index  # Caso 2
courses_some_missing_exams = exam_distribution[
    (exam_distribution / total_students_per_group >= 0.95) & (exam_distribution / total_students_per_group < 1)
].index  # Caso 3
courses_few_exams = exam_distribution[
    (exam_distribution / total_students_per_group <= 0.05) & (exam_distribution / total_students_per_group > 0)
].index  # Caso 4

# Crear la variable "no_exam_data" basado en total_assigned_exams
df["no_exam_data"] = df["total_assigned_exams"].isna().astype(int)

### 2. MANEJO DE ACCESOS

# Lista de variables de acceso
access_variables = [
    "total_time", "average_time", "active_days_count",
    "desktop_sessions_count", "mobile_sessions_count",
    "desktop_imputed_session_time_count", "mobile_imputed_session_time_count",
    "total_sessions_count", "total_imputed_session_time_count", "imputed_ratio"
]

# Crear la variable "missing_access_data" (1 si todas las variables de acceso están ausentes)
df["missing_access_data"] = df[access_variables].isnull().all(axis=1).astype(int)

# Contar registros sin datos de acceso
missing_access_records = df["missing_access_data"].sum()

# Contar grupos completos sin datos de acceso
missing_access_groups = df.groupby("groupKey")["missing_access_data"].mean()
missing_access_groups = (missing_access_groups == 1).sum()  # Contar solo grupos donde todos los registros no tienen acceso

### 3. MANEJO DE FOROS

# Lista de variables de foros
forum_variables = [
    "total_interacciones", "foros_unicos", "primer_interaccion",
    "ultima_interaccion", "rango_tiempo"
]

# Crear la variable "no_forum_data" (1 si todas las variables de foros están ausentes)
df["no_forum_data"] = df[forum_variables].isnull().all(axis=1).astype(int)

# Rellenar variables de conteo con 0 sin usar inplace=True
df["total_interacciones"] = df["total_interacciones"].fillna(0)
df["foros_unicos"] = df["foros_unicos"].fillna(0)
df["rango_tiempo"] = df["rango_tiempo"].fillna(0)

# Mantener "primer_interaccion" y "ultima_interaccion" como NaN
# df["primer_interaccion"].fillna("2000-01-01", inplace=True)  # Esta línea se eliminó para dejar NaN
# df["ultima_interaccion"].fillna("2000-01-01", inplace=True)  # Esta línea se eliminó para dejar NaN

# Contar el número total de cursos sin foros
courses_without_forums = df.groupby("curso")["no_forum_data"].mean()
courses_without_forums = (courses_without_forums == 1).sum()  # Contar solo los cursos donde todos los registros no tienen foros

### 4. IMPRESIÓN DE RESULTADOS

print("\n Distribución de Casos por Curso (Exámenes):")
print(f" Caso 1 - Cursos sin exámenes: {len(courses_without_exams)}")
print(f" Caso 2 - Cursos donde todos tienen exámenes: {len(courses_with_all_exams)}")
print(f" Caso 3 - Cursos con solo 5% alumnos sin exámenes: {len(courses_some_missing_exams)}")
print(f" Caso 4 - Cursos con solo 95% alumnos sin exámenes: {len(courses_few_exams)}")

print("\n Análisis de Datos Faltantes en Accesos:")
print(f" Total de registros individuales sin datos de acceso: {missing_access_records}")
print(f" Total de grupos (groupKey) donde todos los estudiantes no tienen datos de acceso: {missing_access_groups}")

print("\n Análisis de Datos Faltantes en Foros:")
print(f" Total de registros individuales sin datos de foros: {df['no_forum_data'].sum()}")
print(f" Total de cursos (groupKey) sin foros: {courses_without_forums}")

### 5. GUARDAR EL DATASET LIMPIO

df.to_csv(output_path, index=False)
print(f"\n Dataset limpio guardado en: {output_path}")



 Distribución de Casos por Curso (Exámenes):
 Caso 1 - Cursos sin exámenes: 23
 Caso 2 - Cursos donde todos tienen exámenes: 130
 Caso 3 - Cursos con solo 5% alumnos sin exámenes: 30
 Caso 4 - Cursos con solo 95% alumnos sin exámenes: 1

 Análisis de Datos Faltantes en Accesos:
 Total de registros individuales sin datos de acceso: 2266
 Total de grupos (groupKey) donde todos los estudiantes no tienen datos de acceso: 54

 Análisis de Datos Faltantes en Foros:
 Total de registros individuales sin datos de foros: 4489
 Total de cursos (groupKey) sin foros: 145

 Dataset limpio guardado en: /content/drive/MyDrive/Doctorado/Experimentos/Datos/DSv4.0/2. Para unir/merged_cleaned_dataset.csv
